**Author:** Adam McMahan  
**Date Created:** November 4, 2025  
**Version:** 1.0  
**Version Date:** November 4, 2025  

In [18]:
# Libraries

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from textblob import TextBlob
import nltk
import pandas as pd
import ipywidgets as widgets
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from IPython.display import display

In [5]:
# Upload widget for .csv files

upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload CSV File'
)
display(upload_widget)

NameError: name 'widgets' is not defined

In [7]:
# Read uploaded document

def read_uploaded_csv(widget):
    if not widget.value:
        print("No file uploaded.")
        return None

    uploaded_file = list(widget.value.values())[0]
    content = uploaded_file['content']
    df = pd.read_csv(io.BytesIO(content))
    return df

In [11]:
# Column

def create_column_selector(df):
    column_dropdown = widgets.Dropdown(
        options=df.columns.tolist(),
        description='Select Text Column:',
        style={'description_width': 'initial'}
    )
    display(column_dropdown)
    return column_dropdown

def get_text_from_column(df, column_name):
    return ' '.join(df[column_name].dropna().astype(str))

In [32]:
# Sentiment

def filter_sentiment_words(text, sentiment='positive'):
    blob = TextBlob(text)
    words = [word for word in blob.words if len(word) > 2]
    filtered = []
    for word in words:
        polarity = TextBlob(word).sentiment.polarity
        if sentiment == 'positive' and polarity > 0.1:
            filtered.append(word)
        elif sentiment == 'negative' and polarity < -0.1:
            filtered.append(word)
    return ' '.join(filtered)

In [30]:
# Word cloud count

def generate_wordcloud(text, title='Word Cloud', max_words=40):
    wc = WordCloud(
        width=800,
        height=400,
        background_color='white',
        colormap='viridis',
        max_words=max_words
    ).generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=16)
    plt.show()

In [36]:
# User controls

# Sentiment filter dropdown
sentiment_dropdown = widgets.Dropdown(
    options=[('None', None), ('Positive', 'positive'), ('Negative', 'negative')],
    value=None,
    description='Sentiment Filter:',
    style={'description_width': 'initial'}
)

# Word count dropdown
word_count_dropdown = widgets.Dropdown(
    options=[20, 40, 60],
    value=40,
    description='Max Words:',
    style={'description_width': 'initial'}
)

# Generate button
generate_button = widgets.Button(
    description="Generate Word Cloud",
    button_style='success'
)

# Display controls
display(sentiment_dropdown, word_count_dropdown, generate_button)

Dropdown(description='Sentiment Filter:', options=(('None', None), ('Positive', 'positive'), ('Negative', 'neg…

Dropdown(description='Max Words:', index=1, options=(20, 40, 60), style=DescriptionStyle(description_width='in…

Button(button_style='success', description='Generate Word Cloud', style=ButtonStyle())

In [ ]:
# Logic

def on_generate_clicked(b):
    text = read_uploaded_text(upload_widget)
    if not text:
        return

    sentiment = sentiment_dropdown.value
    max_words = word_count_dropdown.value

    if sentiment:
        filtered_text = filter_sentiment_words(text, sentiment=sentiment)
        title = f"{sentiment.capitalize()} Word Cloud"
    else:
        filtered_text = text
        title = "Full Text Word Cloud"

    generate_wordcloud(filtered_text, title=title, max_words=max_words)

generate_button.on_click(on_generate_clicked)